# Firedrake 中的并行计算

## 终端中并行执行 Firedrake 程序

Firedrake 启动并行计算只需在终端中运行以下命令:

```
mpiexec -n <number-of-process> python3 /path/to/your/script.py
```

下面以求解 Poisson 方程的程序为例, 将以下代码保存为文件 `poisson.py`

```python
from firedrake import *

N = 4
test_mesh = RectangleMesh(nx=N, ny=N, Lx=1, Ly=1)
x, y = SpatialCoordinate(test_mesh)
f = sin(pi*x)*sin(pi*y)
g = Constant(0)

V = FunctionSpace(test_mesh, 'CG', degree=1)

u, v = TrialFunction(V), TestFunction(V)
a = inner(grad(u), grad(v))*dx
L = inner(f, v)*dx

bc = DirichletBC(V, g=g, sub_domain='on_boundary')

u_h = Function(V, name='u_h')
solve(a == L, u_h, bcs=bc)
output = VTKFile('data/result.pvd')
output.write(u_h)
```

若使用 2 个进程进行计算, 在激活的 Firedrake 环境中运行:
```
mpiexec -n 2 python3 poisson.py
```

求解结果将保存在 `data/result.pvd` 文件中. 在 `data` 目录下会生成一个 `result` 文件夹,
计算结果保存在该文件夹中, 每个进程对应一个结果文件. `result.pvd` 文件只是这些结果文件的索引.

Firedrake 使用 PETSc 的 DMPlex 来管理网格.
在并行计算中, 计算区域会被划分为多个子区域 (默认情况下区域间有交叠),
并分配给不同的进程, 因此每个进程的结果文件仅包含该进程对应区域的结果.

<!--
```bash
MPICH_NO_LOCAL=1 mpiexec -n 16 -bind-to core -map-by socket python /path/to/script.py
```
-->

### 并行时的一些注意事项


#### 并行时的输出

参考 [py/intro_utils.py](py/intro_utils.py)

1. 第一个进程输出 (其他进程的调用会被忽略)

   ```python
   PETSc.Sys.Print("This is the message will only show once!")
   ```

2. 多个进程同步输出

   + [syncPrint](https://petsc.org/release/petsc4py/reference/petsc4py.PETSc.Sys.html#petsc4py.PETSc.Sys.syncPrint)
   + [syncFlush](https://petsc.org/release/petsc4py/reference/petsc4py.PETSc.Sys.html#petsc4py.PETSc.Sys.syncFlush)

   ```python
   rank, size = COMM_WORLD.rank, COMM_WORLD.size
   PETSc.Sys.syncPrint(f"[{rank}/{size}] This is the message from rank {rank}!")
   if rank == 0:
      PETSc.Sys.syncPrint(f"[{rank}/{size}] Message from rank {rank}!")
   PETSc.Sys.syncFlush()
   ```

#### 仅在指定进程上运算

如果需要在某个特定进程上执行某些操作或计算, 可以使用条件语句. 例如, 只在第 0 号进程上进行绘图操作:

```python
if COMM_WORLD.rank == 0:
    triplot(...)
```

## 在 Jupyter-notebook/Jupyter-lab 中并行执行 Firedrake 代码

在 Jupyter 交互环境中, 我们可以方便地对串行代码进行测试, 从而进行快速验证.
为了在 Jupyter 环境下运行并行程序, `ipyparallel` 包提供了强大的支持,
它能够帮助我们在 Jupyter 中验证并行代码, 同时加深我们对 Firedrake 并行机制的理解.


### 安装配置 `ipyparallel`

1. 激活 Firedrake 环境.
    ```bash
    source /path/to/firedrake/bin/activate
    ```

2. 安装 `ipyparallel`
    
    ```bash
    pip install ipyparallel
    ```

3. 创建 `mpi` 配置文件 (profile)
    
    ```bash
    ipython profile create --parallel --profile=mpi
    ```
    
    运行此命令后，你将看到类似如下的输出
    
    ```bash
    [ProfileCreate] Generating default config file: 
        PosixPath('/home/<your-user-name>/.ipython/profile_mpi/ipython_config.py')
    [ProfileCreate] Generating default config file: 
        PosixPath('/home/<your-user-name>/.ipython/profile_mpi/ipython_kernel_config.py')
    [ProfileCreate] Generating default config file: 
        PosixPath('/home/<your-user-name>/.ipython/profile_mpi/ipcontroller_config.py')
    [ProfileCreate] Generating default config file: 
        PosixPath('/home/<your-user-name>/.ipython/profile_mpi/ipengine_config.py')
    [ProfileCreate] Generating default config file: 
        PosixPath('/home/<your-user-name>/.ipython/profile_mpi/ipcluster_config.py')
    ```

4. 在文件 `.ipython/profile_mpi/ipengine_config.py` 的开头添加以下代码
    
    ```python
    from firedrake import *
    from firedrake.petsc import PETSc
    ```

5. 在 `.ipython/profile_mpi/ipcluster_config.py` 中设置默认引擎 (engines) 为 `mpi`. 

   可以在文件中搜索 `engine_launcher_class`, 并将其编辑为如下内容
    
    ```python
    #    - sshproxy: ipyparallel.cluster.launcher.SSHProxyEngineSetLauncher
    #    - winhpc: ipyparallel.cluster.launcher.WindowsHPCEngineSetLauncher
    #  Default: 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'
    c.Cluster.engine_launcher_class = 'mpi'
    ```

### 测试 `ipyparallel`

启动并连接 `Cluster`.

In [ ]:
import ipyparallel as ipp

cluster = ipp.Cluster(profile="mpi", n=2)
client = cluster.start_and_connect_sync()

在配置好 `ipyparallel` 并启动集群后, 可以通过 Jupyter 中的魔法指令 `%%px` 在多个进程上并行执行代码.
`%%px` 是 `ipyparallel` 提供的魔法命令, 能够在所有并行引擎上同时执行代码.
`--block` 参数用于同步执行代码, 确保代码在所有进程上完成后再继续执行后续命令.

In [ ]:
%%px --block
from firedrake import *
from firedrake.petsc import PETSc
from mpi4py import MPI

N = 4
mesh = RectangleMesh(N, N, 1, 1)
PETSc.Sys.Print("This will only show onece!")

rank, size = mesh.comm.rank, mesh.comm.size
PETSc.Sys.syncPrint(f"[{rank}/{size}] This is the message from rank {rank}!")
PETSc.Sys.syncFlush()
PETSc.Sys.syncFlush()

## 使用 ipyparallel 观察串行和并行过程

### 生成网格并画出网格

#### 串行

In [ ]:
from firedrake import *
from firedrake.pyplot import triplot
from firedrake.petsc import PETSc
from mpi4py import MPI

import matplotlib.pyplot as plt

N = 4
mesh = RectangleMesh(N, N, 1, 1)
mesh.topology_dm.view()
fig, axes = plt.subplots(figsize=[4, 3])
c = triplot(mesh, axes=axes)
xlim = axes.set_xlim([-0.1,1.1])
ylim = axes.set_ylim([-0.1,1.1])

#### 并行

In [ ]:
%%px --block
from firedrake import *
from firedrake.pyplot import triplot
from firedrake.petsc import PETSc
from mpi4py import MPI

import matplotlib.pyplot as plt

N = 4
mesh = RectangleMesh(N, N, 1, 1)
mesh.topology_dm.view()
fig, axes = plt.subplots(figsize=[4, 3])
c = triplot(mesh, axes=axes)
xlim = axes.set_xlim([-0.1,1.1])
ylim = axes.set_ylim([-0.1,1.1])

可以看到上面并行中两个网格图是整体网格的一部分, 且有重叠部分.

### 定义变分问题

#### 串行

In [ ]:
V1 = VectorFunctionSpace(mesh, 'CG', 1)
V2 = FunctionSpace(mesh, 'CG', 2)
W = MixedFunctionSpace([V1, V2])  # W = V1*V2
u1, u2 = TrialFunctions(W)
v1, v2 = TestFunctions(W)
a = dot(u1, v1)*dx + u2*v2*dx

x, y = SpatialCoordinate(mesh)
f = dot(as_vector((sin(x), cos(y))), v1)*dx + cos(y)*v2*dx

bc = DirichletBC(W.sub(0), 0, 1)
uh = Function(W)
problem = LinearVariationalProblem(a, f, uh, bcs=bc)

#### 并行

In [ ]:
%%px --block
V1 = VectorFunctionSpace(mesh, 'CG', 1)
V2 = FunctionSpace(mesh, 'CG', 2)
W = MixedFunctionSpace([V1, V2])  # W = V1*V2
u1, u2 = TrialFunctions(W)
v1, v2 = TestFunctions(W)
a = dot(u1, v1)*dx + u2*v2*dx

x, y = SpatialCoordinate(mesh)
f = dot(as_vector((sin(x), cos(y))), v1)*dx + cos(y)*v2*dx

bc = DirichletBC(W.sub(0), 0, 1)
uh = Function(W)
problem = LinearVariationalProblem(a, f, uh, bcs=bc)

### 函数空间维度

函数空间 V1 和 V2 中的自由度, 可以通过调用函数 dim 得到

In [ ]:
rank, size = mesh.comm.rank, mesh.comm.size
PETSc.Sys.Print(f'Number of dofs of V1: {V1.dim()}')
PETSc.Sys.syncPrint(f'[{rank}/{size}] V1 Node count: {V1.node_count}; V1 Dof count: {V1.dof_count}')
PETSc.Sys.syncFlush()

In [ ]:
%%px --block
rank, size = mesh.comm.rank, mesh.comm.size
PETSc.Sys.Print(f'Number of dofs of V1: {V1.dim()}')
PETSc.Sys.syncPrint(f'[{rank}/{size}] V1 Node count: {V1.node_count}; V1 Dof count: {V1.dof_count}')
PETSc.Sys.syncFlush()

### 节点集和自由度数据集

In [ ]:
PETSc.Sys.syncPrint(f'[{rank}/{size}] V1: {str(V1.node_set)}')
PETSc.Sys.syncPrint(f'[{rank}/{size}]     {repr(V1.node_set)}')
PETSc.Sys.syncPrint(f'[{rank}/{size}] V1: {str(V1.dof_dset)}')
PETSc.Sys.syncPrint(f'[{rank}/{size}]     {repr(V1.dof_dset)}')
PETSc.Sys.syncFlush()

In [ ]:
%%px --block
PETSc.Sys.syncPrint(f'[{rank}/{size}] V1: {str(V1.node_set)}')
PETSc.Sys.syncPrint(f'[{rank}/{size}]     {repr(V1.node_set)}')
PETSc.Sys.syncPrint(f'[{rank}/{size}] V1: {str(V1.dof_dset)}')
PETSc.Sys.syncPrint(f'[{rank}/{size}]     {repr(V1.dof_dset)}')
PETSc.Sys.syncFlush()

#### Size of Set and Data Set

In [ ]:
# Reference:
#   https://github.com/OP2/PyOP2/blob/31471a606a852aed250b05574d1fc2a2874eec31/pyop2/types/set.py#L30
#
# The division of set elements is:
#
#        [0, CORE)
#        [CORE, OWNED)
#        [OWNED, GHOST)
#
# Attribute of dof_dset
#   core_size: Core set size.  Owned elements not touching halo elements.
#   size: Set size, owned elements.
#   total_size: Set size including ghost elements.
#   sizes: (core_size, size, total_size)

In [ ]:
node_set = V1.node_set
msg = f'core size: {node_set.core_size}, size: {node_set.size}, total size: {node_set.total_size}'
# another size: node_set.constrained_size
PETSc.Sys.syncPrint(f'[{rank}/{size}] {msg}')
PETSc.Sys.syncFlush()

In [ ]:
%%px --block
node_set = V1.node_set
msg = f'core size: {node_set.core_size}, size: {node_set.size}, total size: {node_set.total_size}'
PETSc.Sys.syncPrint(f'[{rank}/{size}] {msg}')
PETSc.Sys.syncFlush()

In [ ]:
dof_dset = V1.dof_dset
size_msg = f'core size: {dof_dset.core_size}, size: {dof_dset.size}, total size: {dof_dset.total_size}'
# dim: shape tuple of the values for each element, cdim: product of dim tuple
dim_msg = f'dim: {dof_dset.dim}, cdim: {dof_dset.cdim}'
PETSc.Sys.syncPrint(f'[{rank}/{size}] {size_msg}, {dim_msg}')
PETSc.Sys.syncFlush()

In [ ]:
%%px --block
dof_dset = V1.dof_dset
size_msg = f'core size: {dof_dset.core_size}, size: {dof_dset.size}, total size: {dof_dset.total_size}'
# dim: shape tuple of the values for each element, cdim: product of dim tuple
dim_msg = f'dim: {dof_dset.dim}, cdim: {dof_dset.cdim}'
PETSc.Sys.syncPrint(f'[{rank}/{size}] {size_msg}, {dim_msg}')
PETSc.Sys.syncFlush()

#### ISES of Data Set

In [ ]:
# field_ises:
#   https://github.com/OP2/PyOP2/blob/31471a606a852aed250b05574d1fc2a2874eec31/pyop2/types/dataset.py#L145
#   A list of PETSc ISes defining the global indices for each set in the DataSet.
#   Used when extracting blocks from matrices for solvers.
#
# local_ises:
#   A list of PETSc ISes defining the local indices for each set in the DataSet.
#   Used when extracting blocks from matrices for assembly.
#

In [ ]:
local_ises_msg = f'{[_.getIndices() for _ in W.dof_dset.local_ises]}'
PETSc.Sys.syncPrint(f'[{rank}/{size}] {local_ises_msg}')
PETSc.Sys.syncFlush()

In [ ]:
%%px --block
local_ises_msg = f'{[_.getIndices() for _ in W.dof_dset.local_ises]}'
PETSc.Sys.syncPrint(f'[{rank}/{size}] {local_ises_msg}')
PETSc.Sys.syncFlush()

In [ ]:
field_ises_msg = f'{[_.getIndices() for _ in W.dof_dset.field_ises]}'
PETSc.Sys.syncPrint(f'[{rank}/{size}] {field_ises_msg}')
PETSc.Sys.syncFlush()

In [ ]:
%%px --block
field_ises_msg = f'{[_.getIndices() for _ in W.dof_dset.field_ises]}'
PETSc.Sys.syncPrint(f'[{rank}/{size}] {field_ises_msg}')
PETSc.Sys.syncFlush()

#### Local to Global Map

In [ ]:
# https://gitlab.com/petsc/petsc/-/blob/release/src/binding/petsc4py/src/petsc4py/PETSc/DM.pyx#L1777
# setSection = setLocalSection
# getSection = getLocalSection
# setDefaultSection = setLocalSection
# getDefaultSection = getLocalSection

1. [`firedrake.Halo.local_to_global_numbering`](https://github.com/firedrakeproject/firedrake/blob/ec0329f092b431e8e4c8bd7e41f6667234c9caa3/firedrake/halo.py#L117)
2. [`firedrake.dmcommon.make_global_numbering`](https://github.com/firedrakeproject/firedrake/blob/ec0329f092b431e8e4c8bd7e41f6667234c9caa3/firedrake/cython/dmcommon.pyx#L3307)

In [ ]:
halo = V1.dof_dset.halo
# sf = halo.dm.getPointSF()
# sf.view()
halo.local_to_global_numbering
# halo.dm.getLocalSection().view()
# halo.dm.getGlobalSection().view()

In [ ]:
%%px --block
halo = V1.dof_dset.halo
# sf = halo.dm.getPointSF()
# sf.view()
halo.local_to_global_numbering
# halo.dm.getLocalSection().view()
# halo.dm.getGlobalSection().view()

In [ ]:
# lgmap:
#   https://github.com/OP2/PyOP2/blob/31471a606a852aed250b05574d1fc2a2874eec31/pyop2/types/dataset.py#L111
#   A PETSc LGMap mapping process-local indices to global indices

[W.dof_dset.lgmap.apply(_) for _ in W.dof_dset.local_ises]

In [ ]:
%%px --block
[W.dof_dset.lgmap.apply(_) for _ in W.dof_dset.local_ises]

#### Layout Vector of Data Set

In [ ]:
# dof_dset.layout_vec.getSizes()
vec_msg = f'Local Size: {dof_dset.layout_vec.getLocalSize()}, Size: {dof_dset.layout_vec.getSize()}'
PETSc.Sys.syncPrint(f'[{rank}/{size}] {vec_msg}')
PETSc.Sys.syncFlush()

In [ ]:
%%px --block
# dof_dset.layout_vec.getSizes()
vec_msg = f'Local Size: {dof_dset.layout_vec.getLocalSize()}, Size: {dof_dset.layout_vec.getSize()}'
PETSc.Sys.syncPrint(f'[{rank}/{size}] {vec_msg}')
PETSc.Sys.syncFlush()

### 定义 Solver

In [ ]:
def post_jacobian_callback(X, J, appctx=None):
    # appctx: user data
    # X: vector (gauss value)
    # J: mat
    #
    # mat reference:
    #   https://petsc.org/main/petsc4py/reference/petsc4py.PETSc.Mat.html

    PETSc.Sys.Print("post jacobian callback begin")
    rank, size = J.comm.rank, J.comm.size
    PETSc.Sys.syncPrint(f"  [{rank}/{size}] appctx: {appctx}")
    PETSc.Sys.syncFlush()
    # J.setValueLocal(i, j, 1, PETSc.InsertMode.ADD_VALUES)
    # J.assemble()
    PETSc.Sys.Print("post jacobian callback end")


def post_function_callback(X, F, appctx=None):
    # appctx: user data
    # X: vector (gauss value)
    # F: vector
    #
    # vec reference:
    #   https://petsc.org/main/petsc4py/reference/petsc4py.PETSc.Vec.html

    PETSc.Sys.Print("post function callback begin")
    PETSc.Sys.syncPrint(f"  [{rank}/{size}] appctx: {appctx}")
    PETSc.Sys.syncFlush()
    PETSc.Sys.Print("post function callback end")

appctx = {
    'mesh': mesh,
    'msg': 'The msg from appctx'
}
solver = LinearVariationalSolver(problem,
                                 post_jacobian_callback=partial(post_jacobian_callback, appctx=appctx),
                                 post_function_callback=partial(post_function_callback, appctx=appctx))
solver.solve()

In [ ]:
%%px --block
def post_jacobian_callback(X, J, appctx=None):
    # appctx: user data
    # X: vector (gauss value)
    # J: mat
    #
    # mat reference:
    #   https://petsc.org/main/petsc4py/reference/petsc4py.PETSc.Mat.html

    PETSc.Sys.Print("post jacobian callback begin")
    rank, size = J.comm.rank, J.comm.size
    PETSc.Sys.syncPrint(f"  [{rank}/{size}] appctx: {appctx}")
    PETSc.Sys.syncFlush()
    # J.setValueLocal(i, j, 1, PETSc.InsertMode.ADD_VALUES)
    # J.assemble()
    PETSc.Sys.Print("post jacobian callback end")


def post_function_callback(X, F, appctx=None):
    # appctx: user data
    # X: vector (gauss value)
    # F: vector
    #
    # vec reference:
    #   https://petsc.org/main/petsc4py/reference/petsc4py.PETSc.Vec.html

    PETSc.Sys.Print("post function callback begin")
    PETSc.Sys.syncPrint(f"  [{rank}/{size}] appctx: {appctx}")
    PETSc.Sys.syncFlush()
    PETSc.Sys.Print("post function callback end")

appctx = {
    'mesh': mesh,
    'msg': 'The msg from appctx'
}
solver = LinearVariationalSolver(problem,
                                 post_jacobian_callback=partial(post_jacobian_callback, appctx=appctx),
                                 post_function_callback=partial(post_function_callback, appctx=appctx))
solver.solve()

#### Context of the Solver

In [ ]:
ctx = solver._ctx
PETSc.Sys.syncPrint(f'[{rank}/{size}] Assembler: {ctx._assembler_jac}')
PETSc.Sys.syncPrint(f'[{rank}/{size}] Matrix Size: {ctx._jac.petscmat.getSizes()}')
PETSc.Sys.syncFlush()

In [ ]:
%%px --block
ctx = solver._ctx
PETSc.Sys.syncPrint(f'[{rank}/{size}] Assembler: {ctx._assembler_jac}')
PETSc.Sys.syncPrint(f'[{rank}/{size}] Matrix Size: {ctx._jac.petscmat.getSizes()}')
PETSc.Sys.syncFlush()

#### Matrix 组装

矩阵存储分配相关函数

1. Firedrake method: [ExplicitMatrixAssembler.allocate](https://github.com/firedrakeproject/firedrake/blob/ec0329f092b431e8e4c8bd7e41f6667234c9caa3/firedrake/assemble.py#L1307)

2. PyOp2 class: [Sparsity](https://github.com/OP2/PyOP2/blob/31471a606a852aed250b05574d1fc2a2874eec31/pyop2/types/mat.py#L26)

3. PyOp2 function: [build_sparsity](https://github.com/OP2/PyOP2/blob/31471a606a852aed250b05574d1fc2a2874eec31/pyop2/sparsity.pyx#L121)

4. PyOp2 class: [Mat](https://github.com/OP2/PyOP2/blob/31471a606a852aed250b05574d1fc2a2874eec31/pyop2/types/mat.py#L554)

更多组装细节请看 [矩阵组装内核](./kernels.ipynb)